In [33]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from datetime import datetime

In [34]:
# Read the Global Temperature multisource Excel file and the NOAA sheet
file_path = '/data/keeling/a/sudhansu/i/GlobalTempChange/GlobalTemprature_MultiSource.xlsx'

# Read the data from the Excel file
df_noaa = pd.read_excel(file_path, sheet_name='NOAA', usecols="A:B")
df_crut = pd.read_excel(file_path, sheet_name='HadCRUT_5.0.2.0', usecols="A:B")
df_berkeley = pd.read_excel(file_path, sheet_name='Berkeley-earth-temperature', usecols="A:B")
df_NASA = pd.read_excel(file_path, sheet_name='NASA', usecols="A:B")
df_DCENT = pd.read_excel(file_path, sheet_name='DCENT', usecols="A:B")


# Adjustment = Anerage anomaly 1950-1980
df_base_noaa = df_noaa[(df_noaa['Year'] >= 1850) & (df_noaa['Year'] <= 1900)]
Adjustment_noaa = df_base_noaa['Anomaly(1901-2000 baseline)'].mean()

df_base_crut = df_crut[(df_crut['Time'] >= 1850) & (df_crut['Time'] <= 1900)]
Adjustment_crut = df_base_crut['Anomaly (1961-1990 baseline deg C)'].mean()

df_base_berkeley = df_berkeley[(df_berkeley['Year'] >= 1850) & (df_berkeley['Year'] <= 1900)]
Adjustment_berkeley = df_base_berkeley['Annual_Anomaly(1951-Dec 1980 baseline)'].mean()

df_NASA_1880to1900_avg = df_NASA[(df_NASA['Year'] >= 1880) & (df_NASA['Year'] <= 1900)]['Anomaly(1951-1980 baseline)'].mean()
Adjustment_NASA = df_NASA_1880to1900_avg - 0.03 # As suggested by Gavin Schmidt and Ken Kunkel to bring NASA data to 1850-1900 baseline

df_base_DCENT = df_DCENT[(df_DCENT['Year'] >= 1850) & (df_DCENT['Year'] <= 1900)]
Adjustment_DCENT = df_base_DCENT['GMST (1850ー1900 baseline °C)'].mean()


# Perfor Adjustment to make all data source at 1850-1900 baseline

df_noaa['Anomaly(1850-1900 baseline)'] = df_noaa['Anomaly(1901-2000 baseline)'] - Adjustment_noaa
df_crut['Anomaly(1850-1900 baseline)'] = df_crut['Anomaly (1961-1990 baseline deg C)'] - Adjustment_crut
df_berkeley['Anomaly(1850-1900 baseline)'] = df_berkeley['Annual_Anomaly(1951-Dec 1980 baseline)'] - Adjustment_berkeley
df_NASA['Anomaly(1850-1900 baseline)'] = df_NASA['Anomaly(1951-1980 baseline)'] - Adjustment_NASA
df_DCENT['Anomaly(1850-1900 baseline)'] = df_DCENT['GMST (1850ー1900 baseline °C)'] - Adjustment_DCENT

df_DCENT

,Year,GMST (1850ー1900 baseline °C),Anomaly(1850-1900 baseline)
0,1850,-1.02,-0.181569
1,1851,-0.87,-0.031569
2,1852,-0.84,-0.001569
3,1853,-0.87,-0.031569
4,1854,-0.88,-0.041569
...,...,...,...
169,2019,0.52,1.358431
170,2020,0.54,1.378431
171,2021,0.41,1.248431
172,2022,0.43,1.268431


In [11]:
# Initialize lists to store the coefficients for the quadratic regression
coefficients_noaa = []
window_size = 15
index0_year = 1849
# Loop through the dataframe starting from 1994
for start_year in range(1995, df_noaa['Year'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_noaa = df_noaa[(df_noaa['Year'] >= start_year) & (df_noaa['Year'] <= end_year)]
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_noaa.index + 1, df_window_noaa['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_noaa.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_noaa = pd.DataFrame({
    'Start Year': range(1995, df_noaa['Year'].max() - window_size + 2),
    'End Year': range(1995 + window_size - 1, df_noaa['Year'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_noaa],
    'b': [coeff[1] for coeff in coefficients_noaa],
    'c': [coeff[2] for coeff in coefficients_noaa]
})

# Extract the coefficients for the year 2023
coeffs_2024_noaa = df_quadratic_regression_noaa[df_quadratic_regression_noaa['End Year'] == 2024].iloc[0]
a_2024_noaa = coeffs_2024_noaa['a']
b_2024_noaa = coeffs_2024_noaa['b']
c_2024_noaa = coeffs_2024_noaa['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_noaa = []
for year in range(2025, 2101):
    anomaly = a_2024_noaa * (year - index0_year)**2 + b_2024_noaa * (year - index0_year) + c_2024_noaa
    new_anomalies_noaa.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_noaa = pd.DataFrame(new_anomalies_noaa)

# Append the new anomaly values to the existing df_noaa DataFrame
df_noaa = pd.concat([df_noaa, df_new_anomalies_noaa], ignore_index=True)

# Compute the 11-year centered average
df_noaa['11yr_centered_avg'] = df_noaa['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# Compute the 21-year centered average
df_noaa['21yr_centered_avg'] = df_noaa['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# Compute the 31-year centered average starting at 2010 onwards for df_noaa
df_noaa['31yr_centered_avg'] = df_noaa['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
df_noaa

,Year,Anomaly(1901-2000 baseline),Anomaly(1850-1900 baseline),11yr_centered_avg,21yr_centered_avg,31yr_centered_avg
0,1850,-0.20,-0.026275,NaN,NaN,NaN
1,1851,-0.09,0.083725,NaN,NaN,NaN
2,1852,-0.05,0.123725,NaN,NaN,NaN
3,1853,-0.11,0.063725,NaN,NaN,NaN
4,1854,-0.07,0.103725,NaN,NaN,NaN
...,...,...,...,...,...,...
246,2096,NaN,7.446184,NaN,NaN,NaN
247,2097,NaN,7.573521,NaN,NaN,NaN
248,2098,NaN,7.702021,NaN,NaN,NaN
249,2099,NaN,7.831682,NaN,NaN,NaN


In [17]:
df_quadratic_regression_noaa

,Start Year,End Year,a,b,c
0,1995,2009,-0.000755,0.247720,-19.489524
1,1996,2010,-0.000840,0.277966,-22.113333
2,1997,2011,-0.000637,0.211239,-16.666687
3,1998,2012,-0.000894,0.291683,-22.940191
4,1999,2013,-0.002333,0.747731,-59.069374
5,2000,2014,-0.001112,0.364767,-29.052276
6,2001,2015,0.001618,-0.500055,39.421146
7,2002,2016,0.003615,-1.136605,90.114197
8,2003,2017,0.003503,-1.103777,87.739802
9,2004,2018,0.002315,-0.724656,57.504797


In [12]:
# Initialize lists to store the coefficients for the quadratic regression
coefficients_crut = []
window_size = 15
index0_year = 1849

# Loop through the dataframe starting from 1994
for start_year in range(1995, df_crut['Time'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_crut = df_crut[(df_crut['Time'] >= start_year) & (df_crut['Time'] <= end_year)]
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_crut.index + 1, df_window_crut['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_crut.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_crut = pd.DataFrame({
    'Start Year': range(1995, df_crut['Time'].max() - window_size + 2),
    'End Year': range(1995 + window_size - 1, df_crut['Time'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_crut],
    'b': [coeff[1] for coeff in coefficients_crut],
    'c': [coeff[2] for coeff in coefficients_crut]
})

# Extract the coefficients for the year 2023
coeffs_2024_crut = df_quadratic_regression_crut[df_quadratic_regression_crut['End Year'] == 2024].iloc[0]
a_2024_crut = coeffs_2024_crut['a']
b_2024_crut = coeffs_2024_crut['b']
c_2024_crut = coeffs_2024_crut['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_crut = []
for year in range(2025, 2101):
    anomaly = a_2024_crut * (year - index0_year)**2 + b_2024_crut * (year - index0_year) + c_2024_crut
    new_anomalies_crut.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_crut = pd.DataFrame(new_anomalies_crut)

# Append the new anomaly values to the existing df_crut DataFrame
# df_crut = pd.concat([df_crut, df_new_anomalies_crut], ignore_index=True)
df_crut = pd.concat([df_crut, df_new_anomalies_crut.rename(columns={'Year': 'Time'})], ignore_index=True)

# Compute the 11-year centered average
df_crut['11yr_centered_avg'] = df_crut['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# Compute the 21-year centered average
df_crut['21yr_centered_avg'] = df_crut['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# Compute the 31-year centered average starting at 2010 onwards for df_crut
df_crut['31yr_centered_avg'] = df_crut['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
df_crut

,Time,Anomaly (1961-1990 baseline deg C),Anomaly(1850-1900 baseline),11yr_centered_avg,21yr_centered_avg,31yr_centered_avg
0,1850,-0.417711,-0.061217,NaN,NaN,NaN
1,1851,-0.233350,0.123145,NaN,NaN,NaN
2,1852,-0.229399,0.127096,NaN,NaN,NaN
3,1853,-0.270354,0.086140,NaN,NaN,NaN
4,1854,-0.291521,0.064974,NaN,NaN,NaN
...,...,...,...,...,...,...
246,2096,NaN,8.995113,NaN,NaN,NaN
247,2097,NaN,9.161602,NaN,NaN,NaN
248,2098,NaN,9.329770,NaN,NaN,NaN
249,2099,NaN,9.499618,NaN,NaN,NaN


In [38]:
# Initialize lists to store the coefficients for the quadratic regression
coefficients_berkeley = []
window_size = 15
index0_year = 1849
# Loop through the dataframe starting from 1994
for start_year in range(1995, df_berkeley['Year'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_berkeley = df_berkeley[(df_berkeley['Year'] >= start_year) & (df_berkeley['Year'] <= end_year)]
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_berkeley.index + 1, df_window_berkeley['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_berkeley.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_berkeley = pd.DataFrame({
    'Start Year': range(1995, df_berkeley['Year'].max() - window_size + 2),
    'End Year': range(1995 + window_size - 1, df_berkeley['Year'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_berkeley],
    'b': [coeff[1] for coeff in coefficients_berkeley],
    'c': [coeff[2] for coeff in coefficients_berkeley]
})

# Extract the coefficients for the year 2023
coeffs_2024_berkeley = df_quadratic_regression_berkeley[df_quadratic_regression_berkeley['End Year'] == 2024].iloc[0]
a_2024_berkeley = coeffs_2024_berkeley['a']
b_2024_berkeley = coeffs_2024_berkeley['b']
c_2024_berkeley = coeffs_2024_berkeley['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_berkeley = []
for year in range(2025, 2101):
    anomaly = a_2024_berkeley * (year - index0_year)**2 + b_2024_berkeley * (year - index0_year) + c_2024_berkeley
    new_anomalies_berkeley.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_berkeley = pd.DataFrame(new_anomalies_berkeley)

# Append the new anomaly values to the existing df_berkeley DataFrame
df_berkeley = pd.concat([df_berkeley, df_new_anomalies_berkeley], ignore_index=True)

# Compute the 11-year centered average
df_berkeley['11yr_centered_avg'] = df_berkeley['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# Compute the 21-year centered average
df_berkeley['21yr_centered_avg'] = df_berkeley['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# Compute the 31-year centered average starting at 2010 onwards for df_berkeley
df_berkeley['31yr_centered_avg'] = df_berkeley['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
df_berkeley

,Year,Annual_Anomaly(1951-Dec 1980 baseline),Anomaly(1850-1900 baseline),11yr_centered_avg,21yr_centered_avg,31yr_centered_avg
0,1850,-0.436,-0.129784,NaN,NaN,NaN
1,1851,-0.321,-0.014784,NaN,NaN,NaN
2,1852,-0.292,0.014216,NaN,NaN,NaN
3,1853,-0.295,0.011216,NaN,NaN,NaN
4,1854,-0.283,0.023216,NaN,NaN,NaN
...,...,...,...,...,...,...
246,2096,NaN,9.551245,NaN,NaN,NaN
247,2097,NaN,9.727635,NaN,NaN,NaN
248,2098,NaN,9.905798,NaN,NaN,NaN
249,2099,NaN,10.085732,NaN,NaN,NaN


In [39]:
df_quadratic_regression_berkeley

,Start Year,End Year,a,b,c
0,1995,2009,-0.000687,0.225885,-17.564145
1,1996,2010,-0.000535,0.182672,-14.515733
2,1997,2011,-0.000194,0.073431,-5.779090
3,1998,2012,-0.000550,0.183795,-14.328302
4,1999,2013,-0.002244,0.719706,-56.703626
5,2000,2014,-0.001146,0.375343,-29.702108
6,2001,2015,0.001370,-0.422256,33.469864
7,2002,2016,0.003483,-1.095066,87.021875
8,2003,2017,0.003168,-0.996325,79.280566
9,2004,2018,0.001978,-0.615908,48.908073


In [46]:
# Initialize lists to store the coefficients for the quadratic regression
coefficients_nasa = []
window_size = 15
index0_year = 1849

# Loop through the dataframe starting from 1994
for start_year in range(1995, df_NASA['Year'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_nasa = df_NASA[(df_NASA['Year'] >= start_year) & (df_NASA['Year'] <= end_year)]
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_nasa.index + 1, df_window_nasa['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_nasa.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_nasa = pd.DataFrame({
    'Start Year': range(1995, df_NASA['Year'].max() - window_size + 2),
    'End Year': range(1995 + window_size - 1, df_NASA['Year'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_nasa],
    'b': [coeff[1] for coeff in coefficients_nasa],
    'c': [coeff[2] for coeff in coefficients_nasa]
})

# Extract the coefficients for the year 2024
coeffs_2024_nasa = df_quadratic_regression_nasa[df_quadratic_regression_nasa['End Year'] == 2024].iloc[0]
a_2024_nasa = coeffs_2024_nasa['a']
b_2024_nasa = coeffs_2024_nasa['b']
c_2024_nasa = coeffs_2024_nasa['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_nasa = []
for year in range(2025, 2101):
    anomaly = a_2024_nasa * (year - index0_year)**2 + b_2024_nasa * (year - index0_year) + c_2024_nasa
    new_anomalies_nasa.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_nasa = pd.DataFrame(new_anomalies_nasa)

# Append the new anomaly values to the existing df_NASA DataFrame
df_NASA = pd.concat([df_NASA, df_new_anomalies_nasa], ignore_index=True)

# Compute the 11-year centered average
df_NASA['11yr_centered_avg'] = df_NASA['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# Compute the 21-year centered average
df_NASA['21yr_centered_avg'] = df_NASA['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# Compute the 31-year centered average starting at 2010 onwards for df_NASA
df_NASA['31yr_centered_avg'] = df_NASA['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
df_NASA

,Year,Anomaly(1951-1980 baseline),Anomaly(1850-1900 baseline),11yr_centered_avg,21yr_centered_avg,31yr_centered_avg
0,1850,NaN,NaN,NaN,NaN,NaN
1,1851,NaN,NaN,NaN,NaN,NaN
2,1852,NaN,NaN,NaN,NaN,NaN
3,1853,NaN,NaN,NaN,NaN,NaN
4,1854,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
322,2096,NaN,8.254906,NaN,NaN,NaN
323,2097,NaN,8.400687,NaN,NaN,NaN
324,2098,NaN,8.547853,NaN,NaN,NaN
325,2099,NaN,8.696404,NaN,NaN,NaN


In [48]:
coeffs_2024_nasa

Start Year    2010.000000
End Year      2024.000000
a                0.000692
b               -0.196991
c               14.664851
Name: 15, dtype: float64

In [43]:
# Initialize lists to store the coefficients for the quadratic regression
coefficients_dcent = []
window_size = 15
index0_year = 1849

# Loop through the dataframe starting from 1994
for start_year in range(1994, df_DCENT['Year'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_dcent = df_DCENT[(df_DCENT['Year'] >= start_year) & (df_DCENT['Year'] <= end_year)]
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_dcent.index + 1, df_window_dcent['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_dcent.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_dcent = pd.DataFrame({
    'Start Year': range(1994, df_DCENT['Year'].max() - window_size + 2),
    'End Year': range(1994 + window_size - 1, df_DCENT['Year'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_dcent],
    'b': [coeff[1] for coeff in coefficients_dcent],
    'c': [coeff[2] for coeff in coefficients_dcent]
})

# Extract the coefficients for the year 2023
coeffs_2023_dcent = df_quadratic_regression_dcent[df_quadratic_regression_dcent['End Year'] == 2023].iloc[0]
a_2023_dcent = coeffs_2023_dcent['a']
b_2023_dcent = coeffs_2023_dcent['b']
c_2023_dcent = coeffs_2023_dcent['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_dcent = []
for year in range(2024, 2101):
    anomaly = a_2023_dcent * (year - index0_year)**2 + b_2023_dcent * (year - index0_year) + c_2023_dcent
    print(anomaly)
    new_anomalies_dcent.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_dcent = pd.DataFrame(new_anomalies_dcent)

# Append the new anomaly values to the existing df_DCENT DataFrame
df_DCENT = pd.concat([df_DCENT, df_new_anomalies_dcent], ignore_index=True)

# Compute the 11-year centered average
df_DCENT['11yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# Compute the 21-year centered average
df_DCENT['21yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# Compute the 31-year centered average starting at 2010 onwards for df_DCENT
df_DCENT['31yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
# df_DCENT

1.4450247791423685
1.4677115923291666
1.4893447532858417
1.5099242620123867
1.5294501185088087
1.5479223227751042
1.5653408748112732
1.581705774617319
1.597017022193242
1.6112746175390384
1.6244785606547083
1.6366288515402552
1.6477254901956826
1.65776847662098
1.6667578108161507
1.6746934927811985
1.6815755225161197
1.687403900020918
1.6921786252955897
1.6958996983401349
1.6985671191545606
1.7001808877388562
1.7007410040930289
1.700247468217082
1.6987002801110052
1.6960994397748053
1.6924449472084788
1.6877368024120258
1.6819750053854499
1.6751595561287473
1.6672904546419218
1.6583677009249698
1.6483912949778947
1.6373612368007002
1.625277526393372
1.612140163755921
1.5979491488883468
1.5827044817906462
1.5664061624628225
1.5490541909048687
1.5306485671167955
1.5111892910985958
1.4906763628502695
1.4691097823718202
1.4464895496632515
1.4228156647245527
1.3980881275557273
1.372306938156779
1.345472096527704
1.3175836026685062
1.2886414565791817
1.2586456582597307
1.2275962077101568
1.1

In [44]:
coeffs_2023_dcent

Start Year    2009.000000
End Year      2023.000000
a               -0.000527
b                0.207603
c              -18.751412
Name: 15, dtype: float64

In [36]:

window_size = 15
index0_year = 1849
# Initialize lists to store the coefficients for the quadratic regression
coefficients_dcent = []

# Loop through the dataframe starting from 1994
for start_year in range(1994, df_DCENT['Year'].max() - window_size + 2):
    end_year = start_year + window_size - 1
    df_window_dcent = df_DCENT[(df_DCENT['Year'] >= start_year) & (df_DCENT['Year'] <= end_year)]
    # print(df_window_dcent)
    # Perform quadratic regression
    coeffs = np.polyfit(df_window_dcent.index + 1, df_window_dcent['Anomaly(1850-1900 baseline)'], 2)
    
    # Append the results to the list
    coefficients_dcent.append(coeffs)

# Create a new dataframe to store the results
df_quadratic_regression_dcent = pd.DataFrame({
    'Start Year': range(1994, df_DCENT['Year'].max() - window_size + 2),
    'End Year': range(1994 + window_size - 1, df_DCENT['Year'].max() + 1),
    'a': [coeff[0] for coeff in coefficients_dcent],
    'b': [coeff[1] for coeff in coefficients_dcent],
    'c': [coeff[2] for coeff in coefficients_dcent]
})

# Display the dataframe
# df_quadratic_regression_dcent

# Extract the coefficients for the year 2023
coeffs_2023_dcent = df_quadratic_regression_dcent[df_quadratic_regression_dcent['End Year'] == 2023].iloc[0]
a_2023_dcent = coeffs_2023_dcent['a']
b_2023_dcent = coeffs_2023_dcent['b']
c_2023_dcent = coeffs_2023_dcent['c']

# Compute the anomaly values for the years 2024 to 2100 using the quadratic trend
new_anomalies_dcent = []
for year in range(2024, 2101):
    anomaly = a_2023_dcent * (year - index0_year)**2 + b_2023_dcent * (year - index0_year) + c_2023_dcent
    new_anomalies_dcent.append({'Year': year, 'Anomaly(1850-1900 baseline)': anomaly})

# Convert the list to a DataFrame
df_new_anomalies_dcent = pd.DataFrame(new_anomalies_dcent)

# Append the new anomaly values to the existing df_DCENT DataFrame
df_DCENT = pd.concat([df_DCENT, df_new_anomalies_dcent], ignore_index=True)

# # Compute the 11-year centered average
# df_DCENT['11yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=11, center=True).mean()

# # Compute the 21-year centered average
# df_DCENT['21yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=21, center=True).mean()

# # Compute the 31-year centered average starting at 2010 onwards for df_DCENT
# df_DCENT['31yr_centered_avg'] = df_DCENT['Anomaly(1850-1900 baseline)'].rolling(window=31, center=True).mean()

# Display the updated DataFrame
df_DCENT.to_excel('/data/keeling/a/sudhansu/i/GlobalTempChange/test.xlsx')

In [37]:
df_quadratic_regression_dcent

,Start Year,End Year,a,b,c
0,1994,2008,-0.001343,0.422963,-32.313937
1,1995,2009,-0.001045,0.332591,-25.469555
2,1996,2010,-0.001062,0.342371,-26.570795
3,1997,2011,-0.000487,0.159221,-11.993934
4,1998,2012,-0.000637,0.206013,-15.653857
...,...,...,...,...,...
88,2082,2096,-0.000527,0.207603,-18.751412
89,2083,2097,-0.000527,0.207603,-18.751412
90,2084,2098,-0.000527,0.207603,-18.751412
91,2085,2099,-0.000527,0.207603,-18.751412


In [31]:
df_DCENT['Year'].max()

2100

In [16]:
file_name = f'GlobalTemperature_polyfit_15yrTrend.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df_noaa.to_excel(writer, sheet_name='NOAA', index=False)
    df_crut.to_excel(writer, sheet_name='HadCRUT', index=False)
    df_berkeley.to_excel(writer, sheet_name='Berkeley-Earth-Temperature', index=False)
    df_NASA.to_excel(writer, sheet_name='NASA', index=False)
    df_DCENT.to_excel(writer, sheet_name='DCENT_GlobalTemperature', index=False)